# Bundle adjustment

This notebook visualizes the optimization of 3D points and camera poses using bundle adjustment.

**Subjects covered**  
1. **Definition of bundle adjustment.**
2. **The g2o optimization library.**
3. **Refining 3D point coordinates using bundle adjustment.**
4. **Refining camera poses using bundle adjustment.**
5. **Conclusion**
6. **Sources**

**Why are we interested in these subjects?**      
Previous notebooks demonstrated algorithms for 3D point triangulation and camera pose optimization. These algorithms had **shortcomings**, as they were incapable of optimizing multiple points and poses simultaneously. The triangulation assumed perfect knowledge of poses. The PnP pose estimation assumed perfect knowledge of 3D points. Often we do not have perfect knowledge of either, we only have noisy data and initial guesses. Thus, we need an algorithm that can refine this noisy data to an optimal guess. Bundle adjustment achieves this. 

**Further reading**      
For a comprehensive treatment of bundle adjustment, see chapter eighteen of [Multiple view geometry in computer vision](https://www.robots.ox.ac.uk/~vgg/hzbook/) by Richard Hartley and Andrew Zisserman. For a less thorough but more merciful introduction, see chapter seven of [Computer vision: algorithms and applications](https://szeliski.org/Book/) by Richard Szeliski and the [lectures and shorts](https://www.youtube.com/watch?v=lmj2Jk5tl60&ab_channel=CyrillStachniss) by Cyrill Stachniss.

## Bundle adjustment definition

Bundle adjustment refers to the problem of refining 3D point coordinates and camera poses using noisy 2D images of those points and initial camera pose estimates. Optionally, the camera intrinsics can also be optimized. In other words, it is an algorithm that determines the 3D map of a scene and the cameras' parameters within that scene. The solution is a map and cameras that best explain the observations, namely the cameras' images.

Essentially, bundle adjustment is just an instance of a multi-parameter optimization problem. Similar to the problems of curve fitting and determining the weights of a neural network. As with those problems, bundle adjustment is concerned with minimizing an error metric. The error metric for bundle adjustment is the reprojection error. 

The reprojection error measures the in-image distance between the pixel point in the provided image, and the pixel point from the projected 3D world point. The projected 3D world point is calculated using the estimated camera pose and estimated 3D world point coordinate.

<center>
<img src="./images/reprojection_error.png" width=400 />    
<div align="center">
Image from <a href="https://support.pix4d.com/hc/en-us/articles/202559369-Reprojection-error">Pix4D article</a>
</div>
</center>    


This error can be expressed as a summation over all cameras and points    

$$\begin{align}
E &= \sum_{\text{camera } \in \text{ cameras}} \hspace{0.2em} \sum_{<\text{3D point, 2D point}> \hspace{0.2em} \in \text{ points}} \text{distance}(\text{ 2D point}, \text{ project}(\text{ 3D point }, \text{ camera})) \\\\
E &= \sum_{c \hspace{0.1 em} \in \hspace{0.1 em} C} \hspace{0.2em} \sum_{p, \hspace{0.1 em} p' \hspace{0.1 em} \in \hspace{0.1 em} P} d(p', Q(p, c))) 
\end{align}
$$ 

For a large scene, not all points will be observed by all cameras. If a point is not observed, there is no reprojection error to minimize. Thus, it is useful to add a binary variable $v_{c, \hspace{0.05 em} p}$ to ignore these non-interacting camera-point pairs.

$$ E = \sum_{c \hspace{0.1 em} \in \hspace{0.1 em} C} \hspace{0.2em} \sum_{p, \hspace{0.1 em} p' \hspace{0.1 em} \in \hspace{0.1 em} P} v_{c,\hspace{0.05 em} p}\hspace{0.2 em} d(p', Q(p, c))) $$


#### Multi-parameter optimization 
Bundle adjustment is a multi-parameter optimization problem. There are many multi-parameter optimization schemes one can choose from, such as: 

* iterative gradient descent 
* iterative least-squares (Gauss-Newton)
* iterative gradient descent + least-squares (Levenberg-Marquardt)
* interior-point methods

The goal is to minimize the error by refining the camera poses and point coordinates. Assume we encode camera pose by a unit quaternion $q$ plus a translation vector $c$. Points are encoded as a 3-vector $p$. All these optimization algorithms would calculate the derivatives of $E$ w.r.t. the parameters: $\frac{\partial E}{\partial q_{a}}$, $\frac{\partial E}{\partial q_{b}}$, $\frac{\partial E}{\partial q_{c}}$, $\frac{\partial E}{\partial q_{d}}$, $\frac{\partial E}{\partial c_{x}}$, $\frac{\partial E}{\partial c_{y}}$, $\frac{\partial E}{\partial c_{z}}$, $\frac{\partial E}{\partial t_{x}}$, $\frac{\partial E}{\partial t_{y}}$, $\frac{\partial E}{\partial t_{z}}$. These derivatives would then inform an update rule that iteratively brings the parameters closer to a local optimum.

**Levenberg-Marquardt** is often chosen in the bundle adjustment literature (see implementation and notes in notebook 2). This is because:
* The dampening factor allows balancing between robust gradient descent and fast least-sqaures updates
* There are many sparse implementations available for problems with sparse Jacobians

Why would a sparse implementation be important? Well, applying Levenberg-Marquardt directly to large scenes with many points and cameras would be extremely computationally expensive. Take for example the derivative of the reprojection error w.r.t. a 3D point. Assume this derivative has to be calculated for $100$ thousand points. Now assume that there are $100$ cameras. That means $30$ million derivatives that need to be calculated at each optimization step. Most of these derivatives will be zero, as not all points will be observed by all cameras. Operating on these zeros and keeping them in memory is wasteful. The solution is a sparse implementation. The design of such a sparse algorithm is out of this notebook's scope. For a good summary of such a method, see the paper [SBA: A Software Package for Generic Sparse Bundle Adjustment](http://users.ics.forth.gr/~lourakis/sba/sba-toms.pdf) by Manolis Lourakis and Antonis Argyros with code available [here](https://github.com/balintfodor/sba).

## g2o optimization library

The [g2o optimization library](https://github.com/RainerKuemmerle/g2o) is a framework for optimizing graph-based nonlinear error functions. Here, a graph is a general way to specify a system of coupled variables that produce the error. Formally, each vertex of the graph represents a state variable to optimize, each edge between two variables represents a pairwise
observation of the two vertices it connects. For our bundle adjustment problem, the state variables are world points and camera poses. The edge that connects a point and a camera is the 2D observation of the point by the camera. 

The g2o library defines types for common types of state variables. They a listed here to avoid inline code comments. For cameras, there is the `VertexSCam`, where S stands for 'sparse'. For points, there is `VertexSBAPointXYZ`, which stands for 'Vertex Sparse Bundle Adjustment Point XYZ'. `VertexSCam` defines the camera pose as an `Isometry3d` type. An isometry is a distance-preserving transformation between metric spaces, i.e. a change of coordinate system. This makes sense, as we treat camera poses as coordinate systems in notebook 1. The edge connecting cameras and points is of type `Edge_XYZ_VSC`, where VSC stands for 'Vertex Sparse Camera'. Any mention of `SE3` refers to the mathematical group of rigid body motions, e.g. camera poses. 

We will use [g2opy](https://github.com/uoip/g2opy), a python binding of g2o. For more details on g2o, see the paper [g2o: A General Framework for Graph Optimization](http://ais.informatik.uni-freiburg.de/publications/papers/kuemmerle11icra.pdf). The code documentation is somewhat lacking. The best way to understand its structure is this [system description](https://github.com/uoip/g2opy/blob/master/doc/g2o.pdf) and reading the C++ [source code](https://github.com/RainerKuemmerle/g2o). 

## Imports

In [1]:
from notebook_functions import (camera_centers, 
                                extrinsics, 
                                images, 
                                init_3d_plot, 
                                get_bunny, 
                                project_points_to_picture,
                                intrinsics, 
                                plot_camera_wireframe, 
                                plot_picture, 
                                update_camera_wireframe, 
                                update_picture, 
                                distort_extrinsics, 
                                camera_centers_from_extrinsics)
import ipyvolume as ipv
import numpy as np 
import time 
import g2o 

## Set up a scene and distort bunny 
In this example, we will have perfect knowledge of the camera poses, perfect knowledge of observations, and extremely noisy estimates of 3D points. The goal is to recover the shape of the bunny through optimization.

In [2]:
vis_scale = 5
init_3d_plot()
ipv.zlim(-35, 25)
bunny_coords = get_bunny() 
b_xs, b_ys, b_zs = bunny_coords[:3]
bunny_coords = np.vstack([bunny_coords, np.ones((1, bunny_coords.shape[1]))])
distort_bunny_coords = bunny_coords + np.random.random(bunny_coords.shape) * 5
b_xs, b_ys, b_zs, _ = distort_bunny_coords
scatter_plot = ipv.scatter(b_xs, b_ys, b_zs, size=0.5, marker="sphere", color='lime')
for cam_center, extrinsic, image in zip(camera_centers, extrinsics, images):
    image = np.zeros_like(image)
    image = project_points_to_picture(image, bunny_coords, intrinsics, extrinsic)
    inv_extrinsic = np.linalg.pinv(extrinsic)
    plot_camera_wireframe(cam_center, vis_scale, inv_extrinsic)
    plot_picture(image, inv_extrinsic, vis_scale)
    
ipv.show()

## Define cameras, points, and observations as a g2o graph
Here, we assume all points are visible in all cameras. As can be seen above, this is the case for this simple scene.

In [3]:
# Sets optimizer 
optimizer = g2o.SparseOptimizer()
solver = g2o.BlockSolverSE3(g2o.LinearSolverCSparseSE3())
solver = g2o.OptimizationAlgorithmLevenberg(solver)
optimizer.set_algorithm(solver)

# Set real points in space
true_points = bunny_coords[:3]
distorted_points = distort_bunny_coords[:3]

# Set camera intrinsics
focal_length = (intrinsics[0, 0], intrinsics[1, 1])
principal_point = ((intrinsics[0, 2], intrinsics[1, 2]))
baseline = 0.0 # Not relevant, not a stereo setup
K = np.array([*focal_length, *principal_point, baseline])
g2o.VertexSCam.set_cam(*K)

# Create vertices from cameras and fix them, as their pose is known. 
for idx_extr, extrinsic in enumerate(extrinsics):
    pose = g2o.Isometry3d(np.linalg.inv(extrinsic))
    v_se3 = g2o.VertexSCam()
    v_se3.set_id(idx_extr)
    v_se3.set_estimate(pose)
    if idx_extr == 0:  # Fixing all cameras causes a segmentation fault, so unfix one
        v_se3.set_fixed(False)
    else:    
        v_se3.set_fixed(True)
    v_se3.set_all() # Calculates matrices related to projection
    optimizer.add_vertex(v_se3)

# Each vertex must have a unique id. 
# Ensure point and camera ids do not overlap.
last_pose_id = idx_extr
first_point_id = last_pose_id + 1

# Create vertices from all points and connect them to the cameras
for idx, (distorted_point, true_point) in enumerate(zip(distorted_points.T, true_points.T)):
    point_id = idx + first_point_id
    visible = []
    
    # Project the true points to the images, as observations are perfectly known
    for ext_id in range(len(extrinsics)):
        projected_point = optimizer.vertex(ext_id).map_point(true_point)
        visible.append((ext_id, projected_point))
    
    # Create a point vertex with distorted estimate
    vertex_point = g2o.VertexSBAPointXYZ()
    vertex_point.set_id(point_id)
    # see https://github.com/RainerKuemmerle/g2o/issues/109 on why we use this
    vertex_point.set_marginalized(True)  
    vertex_point.set_estimate(distorted_point)
    optimizer.add_vertex(vertex_point)
    
    # Create an point-camera edge with each camera
    for ext_idx, projected_point in visible:
        edge = g2o.Edge_XYZ_VSC()
        edge.set_vertex(0, vertex_point) 
        edge.set_vertex(1, optimizer.vertex(ext_idx)) 
        edge.set_measurement(projected_point) 
        # The information matrix is the measurement 
        # uncertainty. We set this equal for all measurements.
        edge.set_information(np.identity(3))
        edge.set_parameter_id(0, 0)
        optimizer.add_edge(edge)
    last_point_id = point_id

optimizer.initialize_optimization()
optimizer.set_verbose(False)
print("g2o graph has been created") 

g2o graph has been created


## Visualize point optimization
Visualize 50 steps of point refinement using sparse bundle adjustment optimization.

In [ ]:
def vertex_to_points(optimizer, first_point_id, last_point_id): 
    """ Converts g2o point vertices to their current 3d point estimates. """
    vertices_dict = optimizer.vertices()
    estimated_points = list()
    for idx in range(first_point_id, last_point_id): 
        estimated_points.append(vertices_dict[idx].estimate())
    estimated_points = np.array(estimated_points)
    xs, ys, zs = estimated_points.T
    return xs, ys, zs 

ipv.show()
for i in range(100):
    optimizer.optimize(1)
    xs, ys, zs = vertex_to_points(optimizer, first_point_id, last_point_id)
    scatter_plot.x = xs 
    scatter_plot.y = ys 
    scatter_plot.z = zs 
    time.sleep(0.25)

## Set up a scene and distort cameras 
In this example, we will have perfect knowledge of the bunny points, perfect knowledge of observations, and noisy estimates of camera positions. The goal is to recover camera positions through optimization. Ground truth camera positions are shown in red. Distorted camera positions are shown in blue. 

In [6]:
def vertex_to_extrinsics(optimizer, last_pose_id):
    """ Converts g2o camera vertices to their current 3d camera pose estimates. """
    vertices_dict = optimizer.vertices()
    extrinsics = list()
    for idx in range(last_pose_id + 1): 
        extrinsic = np.linalg.inv(vertices_dict[idx].estimate().matrix())
        extrinsics.append(extrinsic)
    return extrinsics

distorted_extrinsics = [distort_extrinsics(extrinsic, max_angle=30, max_trans=5) for extrinsic in extrinsics]
distorted_extrinsics[0] = extrinsics[0] # Do not distort the first camera
distorted_centers = camera_centers_from_extrinsics(distorted_extrinsics)
vis_scale = 5
init_3d_plot()
ipv.zlim(-35, 25)


bunny_coords = get_bunny() 
b_xs, b_ys, b_zs = bunny_coords
bunny_coords = np.vstack([bunny_coords, np.ones((1, bunny_coords.shape[1]))])
scatter_plot = ipv.scatter(b_xs, b_ys, b_zs, size=0.5, marker="sphere", color='lime')

# Plot ground truth wireframes in red
for cam_center, extrinsic in zip(camera_centers, extrinsics):
    inv_extrinsic = np.linalg.pinv(extrinsic)
    plot_camera_wireframe(cam_center, vis_scale, inv_extrinsic, color='blue')

# Plot the distorted cameras  in blue
extrinsics_estimates = vertex_to_extrinsics(optimizer, last_pose_id)
old_plots = list()
for distort_center, distort_extrinsic, true_extrinsic in zip(distorted_centers, distorted_extrinsics, extrinsics):
    image = np.zeros_like(image)
    image = project_points_to_picture(image, bunny_coords, intrinsics, true_extrinsic) 
    inv_extrinsic = np.linalg.pinv(distort_extrinsic)
    wireframe = plot_camera_wireframe(distort_center, vis_scale, inv_extrinsic, color='red')
    picture = plot_picture(image, inv_extrinsic, vis_scale)
    old_plots.append([wireframe, picture])
    
ipv.show()

## Define cameras, points, and observations as a g2o graph
As with the previous problem, we define a g2o graph.
Ideally, we would have been able to adapt the graph from the first problem and distort and unfix the cameras.
Unfortunately, this leads to a segmentation fault. G2opy probably permanently sets values upon `optimizer.initialize_optimization()`. Thus, at least for now, redefining all vertices seems to be the only option.

In [7]:
optimizer = g2o.SparseOptimizer()
solver = g2o.BlockSolverSE3(g2o.LinearSolverCSparseSE3())
solver = g2o.OptimizationAlgorithmLevenberg(solver)
optimizer.set_algorithm(solver)

for idx_extr, extrinsic in enumerate(extrinsics):
    # Initialize with true intrisics so the 
    # projection images will be correct. 
    # Distorted extrinsic estimates are set at the end 
    pose = g2o.Isometry3d(np.linalg.inv(extrinsic))
    v_se3 = g2o.VertexSCam()
    v_se3.set_id(idx_extr)
    v_se3.set_estimate(pose)
    if idx_extr:
        v_se3.set_fixed(False)
    else: 
        # Fix the first camera
        v_se3.set_fixed(True)
    v_se3.set_all()
    optimizer.add_vertex(v_se3)
    
last_pose_id = idx_extr
first_point_id = last_pose_id + 1

for idx, true_point in enumerate(true_points.T):
    point_id = idx + first_point_id
    visible = []
    # Use true extrinsics to project points
    for ext_id in range(len(extrinsics)):
        projected_point = optimizer.vertex(ext_id).map_point(true_point)
        visible.append((ext_id, projected_point))
    vertex_point = g2o.VertexSBAPointXYZ()
    vertex_point.set_id(point_id)
    vertex_point.set_marginalized(True)
    vertex_point.set_estimate(true_point)
    optimizer.add_vertex(vertex_point)
    for ext_idx, projected_point in visible:
        edge = g2o.Edge_XYZ_VSC()
        edge.set_vertex(0, vertex_point) 
        edge.set_vertex(1, optimizer.vertex(ext_idx)) 
        edge.set_measurement(projected_point) 
        edge.set_information(np.identity(3))
        edge.set_parameter_id(0, 0)
        optimizer.add_edge(edge)
    last_point_id = point_id

# Set distorted extrinsics as pose estimate
for idx_extr, distorted_extrinsic in enumerate(distorted_extrinsics):
    if idx_extr: 
        pose = g2o.Isometry3d(np.linalg.inv(distorted_extrinsic))
        optimizer.vertex(idx_extr).set_estimate(pose)
        optimizer.vertex(idx_extr).set_all()

optimizer.initialize_optimization()
optimizer.set_verbose(False)
print("g2o graph has been created") 

g2o graph has been created


## Visualize camera pose optimization
Visualize 400 steps of camera pose refinement using sparse bundle adjustment optimization.

In [8]:
ipv.show()
time.sleep(2)
for _ in range(50):
    optimizer.optimize(2)
    extrinsics_estimates = vertex_to_extrinsics(optimizer, last_pose_id)
    new_plots = list()
    for idx, (extrinsic, image) in enumerate(zip(extrinsics_estimates, images)):
        wireframe, picture = old_plots[idx]
        cam_center = -extrinsic[:3, :3].T @ extrinsic[:3, 3]
        inv_extrinsic = np.linalg.pinv(extrinsic)
        wireframe = update_camera_wireframe(cam_center, vis_scale, inv_extrinsic, wireframe)
        new_picture = update_picture(image.copy(), inv_extrinsic.copy(), vis_scale, picture)
        new_plots.append([wireframe, new_picture])
    old_plots = new_plots
    time.sleep(2)

## Excercise: Distort bunny, camera poses, and point projections
In the previous cells, we refined the scene and camera locations in isolation.    
As an exercise, create a graph where both camera estimates and scene point estimates are noisy.      
One may even add noise to the coordinates of the projected points.     
Judge the quality of the reconstruction: what is the mean squared error between estimated and true points?   
Does the optimization require more iterations than when refining points or cameras in isolation?    
How "correct" does the reconstruction look?

## Conclusion 
In this notebook we:
* Learned about bundle adjustment 
* Used bundle adjustment to refine scene points and camera locations

Yes, Mr. Frodo, it's over now.    
This concludes the notebook series.    
I hope that you have learned something new about computer vision!   
The reason I started this series was to gain an understanding of the methods that are involved in solving the SLAM problem.    
For the next project, I would like to work on a well-documented implementation of [ORB-SLAM](https://courses.cs.washington.edu/courses/csep576/21au/resources/ORB-SLAM_A_Versatile_and_Accurate_Monocular_SLAM_System.pdf) written in C++, so stay 
tuned if that is of interest to you.   
    
Looking back at this project, I am quite pleased with the notebook visualization format.       
In the future, I would like to continue this series to visualize more classical computer vision subjects, such as:      
* Stereo rectification to create depth maps 
* Camera distortion parameters
* Models of more extreme lenses, such as fisheye and wide-angle  lenses
* Zhang's camera calibration algorithm
* [Alternative camera models](https://www.microsoft.com/en-us/research/uploads/prod/2020/06/Why-Having-10000-Parameters-in-Your-Camera-Model-is-Better-Than-Twelve.pdf) with more parameters for greater accuracy and expressivity
* Dense 3D reconstruction
* Automatic camera calibration using heuristics and visual cues such as angles and straight lines
* 3D point registration such as iterative closest point and derivatives
* Classical optical flow
* The bird's-eye view reprojection of a street as seen in some modern cars [[1]](https://img1.tongtool.com/r/ECEEHGKLCGEFNBEHCGIEMMILJLMJNGLILJGDX0Uk.jpg)[[2]](https://i.ytimg.com/vi/NQYxZ3DTToA/hqdefault.jpg)[[3]](https://img.alicdn.com/imgextra/i3/6000000005662/O1CN01Ph4xT31rhF17bkRxx_!!6000000005662-0-tbvideo.jpg)
* Binary image features, ORB, FAST, and BRIEF
* Modeling 3D lines using Plücker coordinates 


Good luck and stay well!

## Sources
1. [g2o graph optimization framework](https://github.com/RainerKuemmerle/g2o)
2. [Zisserman, Richard Hartley Andrew. "Multiple view geometry in computer vision." 2004.](https://www.robots.ox.ac.uk/~vgg/hzbook/)
3. [Szeliski, Richard. "Computer vision: algorithms and applications." Springer Science & Business Media, 2010](https://szeliski.org/Book/)